In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns',999)
pd.set_option('max_rows',200)

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
sns.set() 
%matplotlib inline 
%config InlineBackend.figure_format = 'retina' 
sns.set_style(style='white')
sns.set_context(context ='paper')

import datetime
import os
from functools import reduce
from tqdm import tqdm_notebook

In [2]:
import selenium as se
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import re

In [3]:
driver = wd.Chrome(r'C:\Users\jeje9\Downloads//chromedriver.exe')

In [4]:
projects = pd.read_csv('projects_fin.csv')

In [5]:
print(projects.shape)
projects.head()

(372572, 23)


,Unnamed: 0,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,state_new,deadline_dt,launched_dt,term,term_str,term_bin,usd_goal_real_bin
0,0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,failed,2015-10-09,2015-08-11,59,59,6,3
1,1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,failed,2017-11-01,2017-09-02,60,60,6,6
2,2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,failed,2013-02-26,2013-01-12,45,45,5,6
3,3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,failed,2012-04-16,2012-03-17,30,30,4,4
4,4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,failed,2015-08-29,2015-07-04,56,56,6,6


In [6]:
df = projects[:2000]

In [7]:
len(df)

2000

In [11]:
# 크롤링 결과 저장할 df
blurb_val_df = pd.DataFrame(columns = ['name', 'blurb', 'state'])

In [9]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_css_selector(css_selector):
    try:
        driver.find_element_by_css_selector(css_selector)
    except NoSuchElementException:
        return False
    return True

def check_not_exists_by_css_selector(css_selector):
    try:
        driver.find_element_by_css_selector(css_selector)
    except NoSuchElementException:
        return True
    return False

In [12]:
driver.get('https://www.kicktraq.com')

for i in tqdm_notebook(range(len(df))):
    name = df['name'][i]
    state = df['state_new'][i]
    # 검색창에 이름 입력하고 엔터 키 누르기
    driver.find_element_by_css_selector('#find').send_keys(name)
    driver.find_element_by_css_selector('#find').send_keys(Keys.ENTER)
    
    # 이름 검색했을 때 안나오는 애들 빼고 + 여러 개 나오는 애들도 빼고
    # 즉 이름이 정확히 일치하는 딱 하나의 프로젝트만 정확하게 나오는 애들로 데이터를 쌓는다
    if check_exists_by_css_selector('#project-info-text') and check_not_exists_by_css_selector('#pagetitle > div.page-subtitle'):
        detail = driver.find_element_by_css_selector('#project-info-text').text
        blurb = detail.split('\n')[0]
        print(str(i) + ' ' + blurb)
        row = [name, blurb, state]
        blurb_val_df = blurb_val_df.append(pd.Series(row, index = ['name', 'blurb', 'state']), ignore_index=True)
    else:
        continue

0 A series of poems about pure love pursued by the passion of another (The Wolf of Elveron)\n\nWhat love is this that so endures........
1 Support artists create short movies of Greetings Form Earth for extraterrestrials, archived in an arts capsule for future first contact
2 Abandoned Property. Sounds of struggle, muffled. Gunshot. Silence.
3 Keeping Great Reggae Music Alive Through RAW Talent!
4 Community Film Project is making a full feature film in Davis County Utah using the abundant talent that exists in the community.
5 Alumni-owned, community-powered craft coffee shop in the heart of Tuscaloosa.
6 Help offset your carbon footprint of coffee. Support Solar Coffee and help save the planet. 1000 lbs Sun Roasted = 117 lbs less CO2
7 Strip. Shot. Enjoy! From portability to shotability, Chaser Strips are revolutionizing alcohol forever.
8 A simple premium audio solution to eliminate the tangle in your life. Extend. Plug. Play. Retract.
11 Fresh new music coming at ya on Lisa Lim's ne

105 Billy Joe a southern waitress with a rough exterior finds herself in the arms of her high school crush at the end of her marriage.
106 PATTY AT THE PARTY is a HD feature length horror movie. Think mumblecore, but with blood and guts. Think early Brian De Palma. Join the party!
107 Ink that makes you think
108 Happy Fix® is looking to start a movement of people across the USA making active decisions to celebrate what's great in life.
109 Become a part of our newest creation and get some one of a kind stuff in return!
110 The Trading Floor On Your Dinning Room Table!
111 The iRNinja is a wireless 4 button keypad that connects to an infrared learning remote base
112 Ebony Child (a musician, writer, filmmaker, and performer) is an independent artist creating a New Album, Comic Book, & Film.
113 Alpha Omega EP Kickstarter
115 creating a show that gives a comedic, inside look to a job not many people know much about
116 We are trying to raise money to go on tour this up coming summer #20

202 More than just an awesome t-shirt: A revolutionary way to support victims of sex trafficking.
203 This is a photo essay to bring the viewer the personality of Cincinnati, neighborhood by neighborhood. 52 neighborhoods in 52 weeks.
205 Move over Montagues and Capulets: here come the Chabadnicks, the Satmers, and the Secular peeps of Brooklyn New York!
206 A Sherlock Holmes tale without Sherlock Holmes. It focuses on Watson and Lestrade trying to solve a strange case in the countryside
207 Intimately autobiographical & poetic, Bea Cordelia's solo show Chasing Blue chronicles her perilous journey toward her trans identity.
210 My son and I want to start our own Custom Shark Rig manufacture. Shark fishing is a big sport in Florida. Tools and supplies needed.
211 L'association FTSL souhaite organiser un troisième opus des conférences Paradigm Shift Meeting. PSM1 eu lieu en 2011 et PSM2 en 2012.
213 Climbing 4 Kidz, the ultimate outdoor adventure
214 Help us share our three-show season w

287 A Ph.D. dissertation on the rescue and rehabilitation of Thoroughbred ex-racehorses and the human(e) communities that support them.
288 Building an awesome blacksmith forge of complete and utter epicness.
289 The project consists in a sweatshirt which hood is implemented with a top opening allowing it to cover both face and neck of the user
290 We roast coffee traditionally, slowly over hardwood charcoal. Amazing Flavor! We want to expand to Certified Farmers Markets
291 I am a singer/songwriter from Houston, TX who moved to Los Angeles, CA with the determination & drive to be SUCCESSFUL!!!
293 Belleflower is a feature film about Sam, a 20-year-old painter, and his descent into schizophrenia.
295 We choose to stand out and inspire. Its time for us to vend at Soundset'14 to connect with the people & just have fun.
297 This is a suspense thriller that will most certainly keep you on the edge of your seat. This is the short "The Get Back" an adaptation to the feature
300 Step by step 

386 Hey, we're Grandkids! We have enough songs to record an LP, and we need your help! We're going to make you proud, promise!
390 The CD project “No Limit” is an open window to the soul of gospel jazz artist Jacquelene "Jackie" Dyles.
391 Are you miserable and talentless? Well so am I! So let's get together and use my poor drawing skills to spread our misery to the world!
392 Artist/Model/Actress/Host\n\nLoving life,
394 Last Man Standing, a card/board game based death match.
395 Our project , put simply, is to make the Dancing Classrooms program widely available in the Greater Richmond VA area.
396 Photographs and stories culled from 10 years of road trips through rural Greece
397 A 6 year old boy struggles with his parents divorce and his mother's growing concern over his claims of battling monsters under his bed
398 Black Knight are the leading suppliers of brick built BBQ kits in the UK and are now offering their products to the US market.
399 Master Keys Pro is a smart new all-in

...

1501 Silver Pickle is a farm to trailer concept. We work with local farms to provide scratch-made food to the people of central Illinois.
1502 A science fiction and fantasy short story service featuring works from Hugo and Dragon Award nominated authors.
1507 Help Angel Road, a unique Country Gospel/Bluegrass sensation, complete album #2--featuring fun originals & beloved gospel favorites.
1509 Exile Saga; an oldstyle 90's JRPG with brand new features. Discover a new world full of fantasy, monsters, lore, angels and demons.
1510 Music started with soul that something you feel. We are bringing you the album Dark Cash. We are bringing you Feelz and Soul.
1511 A tabtag is an individually glowing MacBook sticker using the apple logo as light source. It can be removed and reused many times.
1514 The FUN evolution of dice towers with FREE shipping internationally.
1516 Through one of the most remote corners of the world flows the Lonely Dragon River. This is the story of the people who live 

1614 Respect nature's power to disrupt our everyday living. Prepare for emergencies. We consult, equip and help prepare.
1615 The Freakniks are making their psychedelic freak-folk debut studio album and they need your help.
1616 Somewhere in Time is out to make a new record...with YOU!\nThank you for taking the time to give our project a look.
1617 I podcasten Rollspelsklubben får 3-4 komiker spela det klassiska rollspelet Drakar och Demoner!
1618 A documentary exploring the devastation caused by the human belief that we are separate from animals and nature.
1620 Great novel to fill void left by Vince Flynn passing. Need funding for editor and publisher. 2nd book SCORPION almost done.
1621 We all know not to climb beanstalks, right? In Sondheim's Into the Woods Jnr we join familiar faces in a new kind of fairytale.
1622 Join us to bring healthy, delicious and fun lunches for kids to your area. Convenient for parents, better for kids.
1623 While on the road, a young mother flirts with k

1701 I'm writing my first book, The Bad Seed. It is the first in my Battle for the Heavens series. Help me and lets make magic! .
1702 Promoting education and enrichment in our community, we organize fundraising events for the cheese industry's non-profit organizations.
1703 Rennys Oki Doki food truck is almost ready to launch. The only thing standing between you and some delicious Okinawan food is artwork!
1704 Imagine The Golden Girls summering on Fantasy Island if it were run by Patsy & Edina!
1705 Book exploring the composition and construction of Sonnenzimmer's screen printed posters.
1707 The battle of Dallas is an instant ,and strategic card game.Every game round has an unexpected ending.
1708 Creating handmade quality leather accessories of all types. From earrings, bracelets, and necklaces to belts, money clips and holsters.
1709 We are a new micro game studio based in Largs, Scotland, UK. This project is the first part of a new Trilogy RPG we are working on.
1712 Explore a ma

1805 Pink Crush is a whimsical cake and cupcake business!
1807 An open source instruction and plan set from the digital design model & files of this working 5 axis CNC machine.
1808 An intimate documentary about the lifestyle of three communities in Romania who have been self-sufficient for centuries.
1809 Kerry Lyn Dalton was convicted in 1995 of a torture/murder and is currently on Death Row at the California Womens Correctional facility
1811 Only with your support can we take this wonderful program to the next level. With music we can make this happen!
1813 "Bookends: Vol. 1, 2, 3" explores the world of life and death through music, art, and literature.
1814 Bring music and sound responsiveness into all your Arduino projects easily, with the MSGEQ7 Breakout Board!
1817 Albert Einstein is revealed as a firebrand; a fearless, influential social and political activist and humanitarian.
1818 Edge Goalkeeping is a new goalkeeper glove brand to come. Sports equipement project, funds neede

1918 This gem is unlike any other on Earth. I think it is more sparkly than any other blue gem. It is also a very hard, durable gem!
1919 EL EVENTO tells the story of the AWF, an independent live painting tournament from Puerto Rico inspired by Professional Wrestling.
1922 The nail clipper reinvented! Providing more safety, more grip and allowing to clip nails at the desired length, with less filing.
1923 By photographing a medical volunteer trip, Journey to Jacmel intends to capture the spirit, hope and beauty of this Haitian community.
1924 Women, we are not intended to be perfect, but to be whole, and Whole Women Win! 9 messages to encourage women to live their best lives.
1925 the feath3r theory completes Andy Warhol’s vision of a remake for the1985 movie-musical classic 'A CHORUS LINE'.
1927 My beloved bike was stolen this holiday season, so I am taking it as a sign from the universe to learn to build a bike.
1928 Foldable | Great for narrow and delicate collars | Saves space | Po

In [13]:
blurb_val_df.head(20)

,name,blurb,state
0,The Songs of Adelaide & Abullah,A series of poems about pure love pursued by t...,failed
1,Greeting From Earth: ZGAC Arts Capsule For ET,Support artists create short movies of Greetin...,failed
2,Where is Hank?,"Abandoned Property. Sounds of struggle, muffle...",failed
3,ToshiCapital Rekordz Needs Help to Complete Album,Keeping Great Reggae Music Alive Through RAW T...,failed
4,Community Film Project: The Art of Neighborhoo...,Community Film Project is making a full featur...,failed
5,Monarch Espresso Bar,"Alumni-owned, community-powered craft coffee s...",success
6,Support Solar Roasted Coffee & Green Energy! ...,Help offset your carbon footprint of coffee. S...,success
7,Chaser Strips. Our Strips make Shots their B*tch!,Strip. Shot. Enjoy! From portability to shotab...,failed
8,SPIN - Premium Retractable In-Ear Headphones w...,A simple premium audio solution to eliminate t...,failed
9,Lisa Lim New CD!,Fresh new music coming at ya on Lisa Lim's new...,success


In [15]:
len(blurb_val_df)

1348

In [14]:
blurb_val_df.to_csv('blurb_validation_df.csv')